In [1]:
!ls /dev/ttyUSB*

/dev/ttyUSB0  /dev/ttyUSB1


In [2]:
%load_ext autoreload
%autoreload 2

import sys, re

# https://github.com/fzinfz/meter-dlt645
sys.path.append("/data/powermeter_dlt645/meter-dlt645")

from test_dlt645 import *
import test_dlt645

# https://github.com/fzinfz/flask-DLT645
sys.path.append("/data/flask-DLT645/lib")

from read import *
from write import *

In [3]:
chn=dlt645.Channel(port_id = '/dev/ttyUSB1', tmo_cnt = 10, wait_for_read = 0.5)
chn.open()

True

# auto detect address

In [4]:
read_meter_address(chn, verbose=1)
addr = chn.rx_addr
print(addr)


--- Read meter address with broadcase mode ---
tx_frame   : fe fe 68 aa aa aa aa aa aa 68 13 00 df 16 
tx_addr    : aa aa aa aa aa aa 
tx_ctrl    : 13 
tx_payload : 
wait...

rx_frame   : 68 69 85 31 28 01 01 68 93 06 9c b8 64 5b 34 34 2d 16 
rx_addr    : 01 01 28 31 85 69 
rx_ctrl    : 93 
rx_payload : 69 85 31 28 01 01 

Meter address: 010128318569
[1, 1, 40, 49, 133, 105]


In [5]:
# algorithm for converting back to human readable
''.join([ '{0:0>2x}'.format(s) for s in addr ])

'010128318569'

# manually set address

In [6]:
addr = [ int(s,16) for s in re.findall('..', '001522454104') ]

# Read meter
restart kernel if data incorrect

In [6]:
m = Meter(chn, addr, level=4, verbose=0)
m.read_meter()

{'内部电池电压': (3.69, 'V'),
 '电能-组合有功总-上结算日': (0.0, 'kWh'),
 'A相电压': (227.9, 'V'),
 'A相电流': (0.357, 'A'),
 '功率-一分钟有功总平均': ('48.00', 'W'),
 '功率因数-总': (0.575, ''),
 '功率-瞬时总有功': ('46.80', 'W'),
 '电能-组合有功总-当前': (0.52, 'kWh'),
 '功率-A相': ('81.36', 'W'),
 '电能:本周期': ('0.52', 'kWh'),
 '日期时间': ('20201129 20:43:33', '')}

In [7]:
m = Meter(chn, addr, level=1, verbose=1)
m.read_meter()

tx_frame   : fe fe 68 69 85 31 28 01 01 68 11 04 33 33 33 33 fa 16 
tx_addr    : 01 01 28 31 85 69 
tx_ctrl    : 11 
tx_payload : 00 00 00 00 
wait...

rx_frame   : 68 69 85 31 28 01 01 68 91 08 33 33 33 33 85 33 33 33 9c 16 
rx_addr    : 01 01 28 31 85 69 
rx_ctrl    : 91 
rx_payload : 00 00 00 00 52 00 00 00 



{'电能-组合有功总-当前': (0.52, 'kWh')}

In [10]:
passwd

[4, 0, 0, 0]

In [8]:
# re-run if IndexError

verbose=0
read_last_outage_timestamp(chn, addr, 1, verbose)

for m in range(0,2):
    for n in [0,2,4]:
        read_energy(chn, addr, m, n, verbose)


--- Read last outage timestamp of N = 1 ---
Last outage timestamp: 20-11-28 00:27:31 ----> 20-11-28 00:50:26 

--- Read energy (Current Month) ---
Total: 0.52 kWh

--- Read energy (Current Month) ---
Peak: 0.35 kWh

--- Read energy (Current Month) ---
Trough: 0.17 kWh

--- Read energy (Last 1 Month) ---
Total: 0.00 kWh

--- Read energy (Last 1 Month) ---
Peak: 0.00 kWh

--- Read energy (Last 1 Month) ---
Trough: 0.00 kWh


# change Date & Time
https://github.com/glx-technologies/meter-dlt645  
运行前开前挡板按编程键

    rx_ctrl    : 
        94  => success
        d4  => fail
        91  => read success

In [46]:
!date

2020年 11月 01日 星期日 12:37:15 CST


In [17]:
change_date(chn, addr)

201129
[32, 17, 41]
tx_frame   : fe fe 68 04 41 45 22 15 00 68 14 10 34 34 33 37 37 33 33 33 33 33 33 33 33 5c 44 53 49 16 
tx_addr    : 00 15 22 45 41 04 
tx_ctrl    : 14 
tx_payload : 01 01 00 04 04 00 00 00 00 00 00 00 00 29 11 20 
wait...

rx_frame   : 68 04 41 45 22 15 00 68 91 08 34 37 33 33 7b 34 33 33 10 16 
rx_addr    : 00 15 22 45 41 04 
rx_ctrl    : 91 
rx_payload : 01 04 00 00 48 01 00 00 



In [29]:
passwd

[4, 17, 17, 17]

In [28]:
change_time(chn, addr)

193241
[25, 50, 65]
tx_frame   : fe fe 68 69 85 31 28 01 01 68 14 0f 35 34 33 37 37 33 33 33 33 33 33 33 74 65 4c d0 16 
tx_addr    : 01 01 28 31 85 69 
tx_ctrl    : 14 
tx_payload : 02 01 00 04 04 00 00 00 00 00 00 00 41 32 19 
wait...

rx_frame   : 68 69 85 31 28 01 01 68 d4 01 37 25 16 
rx_addr    : 01 01 28 31 85 69 
rx_ctrl    : d4 
rx_payload : 04 



# close channel

In [52]:
chn.close()